# Generar datos en DB Vectorial

In [21]:
import streamlit as st
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import SpiderLoader, PyPDFLoader, WebBaseLoader, DirectoryLoader
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
import os


### Prepara entorno para generar datos

In [22]:
FAISS_PATH = "/Users/rdiaz/OneDrive/10_Codigofacilito/03_Bootcamp_llms/10_project/sb_llm_project/faiss_data"
DIR_PATH_TXT = "/Users/rdiaz/OneDrive/10_Codigofacilito/03_Bootcamp_llms/10_project/sb_llm_project/data/txt"
DIR_PATH_CSV = '/Users/rdiaz/OneDrive/10_Codigofacilito/03_Bootcamp_llms/10_project/sb_llm_project/data/csv'

os.environ['GOOGLE_API_KEY'] = st.secrets["google_gemini"]["api_key_gemini"]

### Genera loaders que cargan datos de PDFs

In [23]:
#loader_txt = DirectoryLoader(DIR_PATH_TXT, glob="*.txt")
#loader_csv = DirectoryLoader(DIR_PATH_CSV, glob="*.csv")
loader_web = WebBaseLoader(["https://secopml.super.site/", "https://secopml.super.site/modelado-y-evaluacin", "https://secopml.super.site/detalle-del-proceso-de-despliegue"])
loader_pdf = PyPDFLoader("data/pdf/Proyecto_ML-SECOP.pdf", extract_images=False)


### Genera Chunks de los documentos
Realiza merge de los loaders de los documentos y genera chunks

In [24]:
loader_docs = MergedDataLoader(loaders=[loader_pdf, loader_web])
documents = loader_docs.load()
docs_processed = filter_complex_metadata(documents)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)
chunks = text_splitter.split_documents(documents)

### Genera datos en DB Vectorial
Genera Embedding con Gemini y guarda datos en la base de datos vectorial FAISS

In [25]:
db = FAISS.from_documents(chunks, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
db.save_local(FAISS_PATH)

I0000 00:00:1723063943.092743   19216 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
